## Algoritmo de busqueda
generalizado para comandos del tipo:

In [ ]:
#cmd="CALCulate:BANDwidth:PTPeak"
#cmd="CALCulate:MARKer<n>:PTPeak"
#cmd="CALCulate:MARKer:FCOunt:RESolution"
cmd="CALCulate:NTData[:STATe]"
def busqueda(cmd):

    cmd_s=cmd.split(":")
    class1=cmd_s[0].split("[")[0].split("]")[0].lower()
    class2=cmd_s[1].split("[")[0].split("]")[0].lower()
    
    class2=class2.split("<")
    if (len(class2)>1):
        op=class2[1].split(">")[0]
        op=op.split("[")[0].split("]")[0]
        class2=class2[0].split("[")[0].split("]")[0]
    else:
        class2=class2[0]
        op=0
        
        
    if (len(cmd_s)>3):
        func=cmd_s[2].split("[")[0].split("]")[0].lower()
        for i in range(3,len(cmd_s),1):
            func=func+"_"+cmd_s[i].split("[")[0].split("]")[0].lower()
    else:
        func=cmd_s[2].split("[")[0].split("]")[0].lower()

    return(class1,class2,func,op)
busqueda(cmd)

## generación de archivo
> Requiere la apertura y el cierre de forma externa

In [ ]:
static_c2=[]
def file_gen(cmd,tipo,class1,class2,funcion,op,filename):
    f = open (filename,'a')
    
    if(op!=0):
        cmd=cmd.split("<")
        cmd=cmd[0]+"\"+str(op)+\""+cmd[1].split(">")[1]


    if (class2):
        # clase de jerarquia 2
        f.write("\nclass {}:\n\tdef __init__(self):\n".format(class2))
        f.write("\t\tINFO_NO_PARAM=\"no admite parametro, se envia consulta\"\n")
        static_c2.append(class2)    
        

    f.write("\tdef {}(*n):\n\t\tif len(n) == 0:\n".format(funcion))    
    if (tipo=="WQ"):
        f.write("\t\t\tinst.query(\"{}?\")\n\t\telse:\n\t\t\tinst.write(\"{} \" +str(n))\n".format(cmd,cmd))
    elif (tipo=="Q"):
        f.write("\t\t\tinst.query(\"{}?\")\n\t\telse:\n\t\t\tinst.query(\"{} \" +str(n))\n".format(cmd,cmd))
    elif (tipo=="W"):
        f.write("\t\t\tinst.write(\"{}?\")\n\t\telse:\n\t\t\tinst.write(\"{} \" +str(n))\n".format(cmd,cmd))
    else:
        f.write("\t\t\tinst.read(\"{}?\")\n\t\telse:\n\t\t\tinst.read(\"{} \" +str(n))\n".format(cmd,cmd))
        
    # clase de jerarquia 1, ultima en generarse
    if(class1): # seteado en el loop luego de la primer pasada (no está la clase principal)
        f.write("class {}:\n\tdef __init__(self):\n".format(class1))
        for c in static_c2:
            f.write("\t\tself.{}={}()\n".format(c,c))
    f.close()
    

## vector de comandos
comandos puestos en un vector aclarando su tipo
- WQ: write & query
- Q: query
- W: write
- R: read
- Z: sin parametro ni rta

Formato: [cmd1,type,cmd2,type....,cmdn,type]

# Proceso general
Se recorre el vector y se va realizando la busqueda y la escritura

In [ ]:

def commad2file(cmdV,filename):
    rango = len(cmdV)
    class2=[0]
    
    for i in range(0,rango,2):
        cmd=cmdV[i]
        tipo=cmdV[i+1]

        (c1,c2,funcion,op)=busqueda(cmd)

        class2.append(c2)
        if (c2==class2[-2]):
            c2=0



        if(i==rango-2):
            file_gen(cmd,tipo,c1,c2,funcion,op,filename)
        else:
            file_gen(cmd,tipo,0,c2,funcion,op,filename)


# Levantando archivo de comandos
Se espera un archivo del tipo:

> tipo:comando



In [ ]:
def instrument_class(lista_de_comandos,salida):
    f=open(lista_de_comandos,"r")
    cmdV=[]
    str = f.readline()
    while (str):
        tipo=str.split(":")[0]
        index=str.find(":")+1
        cmd=str[index:].split("\n")[0]
        cmdV.append(cmd)
        cmdV.append(tipo)
        str = f.readline()


    if os.path.exists(salida):
        os.remove(salida)
    commad2file(cmdV,salida)
    print("clase para manejo del instrumental creada")

# Modo de uso

se indica:

- archivo que contiene el listado de comandos
- archivo de salida

Luego:
```py
import salida
```

In [ ]:
import os

lista_de_comandos="classRigol.py"
salida="commans_rigol815.py"

instrument_class(lista_de_comandos,salida)

In [ ]:
from commans_rigol815 import calculate 
rigol=calculate()

# Ideas de mejoras:
- incluir la deteccion de comandos sin parametro
- detectar los distintos tipos de comandos (ej, estructura sence, :FETCh:HARMonics:AMPLitude <n>, etc) y considerarlos por separado
- hacer el proceso como venimos haciendo y por cada ## (ver lista rigol 815) pasar nuevamente para agregar otra clase
- al final del archivo, hacer la clase Rigol_815 final

## Pequeña utilidad para facilitar edición

In [ ]:
lista_de_comandos="lista_rigol_815.py"
salida="lista_rigol_815_identificada.py"

f=open(lista_de_comandos,"r")
s=open(salida,"w")
cmdV=[]
str = f.readline()
while (str):
    print(str)
    if(str[0]==":"):
        tipo=input("especifique tipo: WQ, Q, W, R o Z. EXIT para finalizar\n")
        if(tipo == "EXIT"):
            break
        s.write(tipo+str)
    else:
        s.write(str)
    str = f.readline()

f.close()
s.close()